In [52]:
# Import several packages and modules
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.datasets import CIFAR100
from torch.utils.data import DataLoader
import torchvision.transforms as tt

In [53]:
#Set hyper parameters
in_channels = 3
num_classes = 100
learning_rate = 0.001
batch_size = 512
num_epochs = 30

train_acc_history = []
train_loss_history = []
val_acc_history = []
val_loss_history = []

In [54]:
#Load Dataset (CIFAR-100)
train_ds = CIFAR100(
    root='./data', train=True, download=True, transform=tt.ToTensor())
val_ds = CIFAR100(
    root='./data', train=False, download=True, transform=tt.ToTensor())

train_dl = DataLoader(dataset=train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(dataset=val_ds, batch_size=batch_size*2, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [55]:
#Define BasicBlock Module
class block(nn.Module):
    def __init__(
        self, in_channels, intermediate_channels, identity_downsample=None, stride=1):
        super(block, self).__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(
            in_channels, intermediate_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(intermediate_channels)
        self.conv2 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False)
        self.bn2 = nn.BatchNorm2d(intermediate_channels)
        self.conv3 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels * self.expansion,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False)
        self.bn3 = nn.BatchNorm2d(intermediate_channels * self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity = x.clone()

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x

#Define ResNet model
class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Essential ResNet architecture
        self.layer1 = self._make_layer(
            block, layers[0], intermediate_channels=64, stride=1)
        self.layer2 = self._make_layer(
            block, layers[1], intermediate_channels=128, stride=2)
        self.layer3 = self._make_layer(
            block, layers[2], intermediate_channels=256, stride=2)
        self.layer4 = self._make_layer(
            block, layers[3], intermediate_channels=512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x

    def _make_layer(self, block, num_residual_blocks, intermediate_channels, stride):
        identity_downsample = None
        layers = []

        """To half the input space for ex, 56x56 -> 28x28 (stride=2), or channels changes
        we need to adapt the Identity (skip connection) so it will be able to be added
        to the layer that's ahead"""
        if stride != 1 or self.in_channels != intermediate_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    intermediate_channels * 4,
                    kernel_size=1,
                    stride=stride,
                    bias=False),
                nn.BatchNorm2d(intermediate_channels * 4))

        layers.append(
            block(self.in_channels, intermediate_channels, identity_downsample, stride))

        # The expansion size is always 4
        self.in_channels = intermediate_channels * 4

        # For example for first resnet layer: 256 will be mapped to 64 as intermediate layer,
        # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
        # and also same amount of channels.
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, intermediate_channels))

        return nn.Sequential(*layers)


def ResNet50(img_channel=3, num_classes=1000):
    return ResNet(block, [3, 4, 6, 3], img_channel, num_classes)

In [56]:
#Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNet50(img_channel=in_channels, num_classes=num_classes).to(device)

In [57]:
# loss and optimizer
criterion = F.cross_entropy
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [58]:
# Set model, Train/Evaluate
def check_train_accuracy(train_dl, model):
  
  model.eval()

  with torch.no_grad():
    for x, y in train_dl:
      x = x.to(device=device)
      y = y.to(device=device)

      score = model(x)
      _, pred = torch.max(scores, dim=1)
      t_acc = torch.tensor(torch.sum(pred == labels).item()/len(pred))

    train_acc_history.append(t_acc)
  
  model.train()

In [59]:
# check accuracy
def check_val_accuracy(val_dl, model):
  
  model.eval()

  with torch.no_grad():
    for x, y in val_dl:
      x = x.to(device=device)
      y = y.to(device=device)

      score = model(x)
      _, pred = torch.max(scores, dim=1)
      v_acc = torch.tensor(torch.sum(pred == labels).item()/len(pred))

    print('val_accuracy {: .2f}%'.format(v_acc*100))
    val_acc_history.append(v_acc)
  
  model.train()

In [60]:
#Run
%%time
for epoch in range(num_epochs):
  for batch_idx, (data, labels) in enumerate(train_dl):
    # send data to gpu
    data = data.to(device=device)
    labels = labels.to(device=device)
    # forward pass
    scores = model(data)
    # calculate loss
    train_loss = criterion(scores, labels)
    # backward pass
    train_loss.backward()
    # update weight
    optimizer.step()
    # set gradient back to zero
    optimizer.zero_grad()
    train_loss_history.append(train_loss)
        
  for batch_idx, (data, labels) in enumerate(val_dl):
    # send data to gpu
    data = data.to(device=device)
    labels = labels.to(device=device)
    # forward pass
    scores = model(data)
    # calculate loss
    val_loss = criterion(scores, labels)
    # backward pass
    val_loss.backward()
    # update weight
    optimizer.step()
    # set gradient back to zero
    optimizer.zero_grad()
    val_loss_history.append(val_loss)

  print('[{}]'.format(epoch),end=' ')
  print('train_loss {: }, '.format(train_loss.item()) ,end=' ')
  print('val_loss {: }, '.format(val_loss.item()) ,end=' ')
  check_val_accuracy(val_dl, model)

[0] train_loss  3.7840797901153564,  val_loss  3.427515745162964,  val_accuracy  17.86%


KeyboardInterrupt: ignored

In [61]:
#if torch.cuda.is_available():

    #device = 'cpu'

#print(device)


In [64]:
input_parameter = next(model.parameters())
input_parameter.size()

torch.Size([64, 3, 7, 7])

In [70]:
#Accuracy plot
#no numeric data to plot
#can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

import torch
import numpy as np
val_acc_series = pd.Series(val_acc_history)
train_acc_series = pd.Series(train_acc_history)
#val_acc_series = pd.Series(torch.stack(val_acc_history))
#train_acc_series = pd.Series(torch.stack(train_acc_history))

val_acc_series.plot(label='val')
train_acc_series.plot(label='train')
plt.title('ResNet Accuracy')
plt.ylabel('Accuracy(%)')
plt.xlabel('Epoch')
plt.savefig('./ResNet50_CIFAR100_acc.png', dpi=500)
plt.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys


TypeError: ignored

In [66]:
#Loss plot
val_loss_series = pd.Series(torch.stack(train_loss_history))
train_loss_series = pd.Series(torch.stack(train_loss_history))

val_loss_series.plot(label='val')
train_loss_series.plot(label='train')
plt.title('ResNet Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.savefig('./ResNet_loss.png', dpi=500)
plt.show()

TypeError: ignored

In [67]:
#Save
torch.save(model.state_dict, 'ResNet50_CIFAR100.pth')